# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from pprint import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_path = "../output_data/cities.csv"
city_df = pd.read_csv(city_path)
# city_df.sort_values("Max Temp",ascending=False)
city_df.sort_values("Humidity",ascending=False)
len(city_df["Humidity"])

540

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)
locations1 = city_df[["Lat","Lng"]].astype(float)
ratings = city_df["Humidity"].astype(float)

fig = gmaps.figure(center = (10,0),zoom_level=2)
heat_layer1 = gmaps.heatmap_layer(locations1,weights=ratings,dissipating=False,max_intensity=max(city_df["Humidity"]),point_radius=3)
fig.add_layer(heat_layer1)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrowed_df = city_df.loc[(city_df["Max Temp"]<80)&(city_df["Max Temp"]>70)&(city_df["Wind Speed"]<10)&(city_df["Cloudiness"]==0)]
# len(narrowed_df["City"])
hotel_df = narrowed_df.dropna(how="any").reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.79,-50.01,70.48,54,0,3.60,BR,1595262820
1,longkou,37.65,120.33,74.28,77,0,4.92,CN,1595262839
2,ponta do sol,32.67,-17.10,73.99,60,0,0.98,PT,1595262852
3,malibu,34.01,-118.81,75.00,85,0,0.89,US,1595262681
4,longonjo,-12.91,15.25,73.40,28,0,0.90,AO,1595262866
5,bukama,-9.20,25.85,73.74,29,0,1.00,CD,1595262870
6,bingol,38.88,40.49,77.11,16,0,0.70,TR,1595262870
7,namibe,-15.20,12.15,71.22,59,0,3.58,AO,1595262873
8,san quintin,30.48,-115.95,76.55,53,0,8.61,MX,1595262875
9,surt,31.21,16.59,79.50,65,0,5.96,LY,1595262875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df["Hotel Name"] = ""
params = {
    "radius":5000,
    "types":"lodging",
    "key word": "hotel",
    "key": g_key
}

for i,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_json = requests.get(base_url,params=params).json()
#     pprint(hotel_json)
    try: 
        hotel_df.loc[i,"Hotel Name"] = hotel_json["results"][0]["name"]
    except (KeyError,IndexError):
        print("Missing field/result... skipping.")
    
        
    

Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=''].reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,castro,-24.79,-50.01,70.48,54,0,3.60,BR,1595262820,CHACARA BAILLY
1,longkou,37.65,120.33,74.28,77,0,4.92,CN,1595262839,Pai Hotel Yantai Longkou City Heping Road
2,ponta do sol,32.67,-17.10,73.99,60,0,0.98,PT,1595262852,Hotel do Campo
3,malibu,34.01,-118.81,75.00,85,0,0.89,US,1595262681,Malibu Country Inn
4,bingol,38.88,40.49,77.11,16,0,0.70,TR,1595262870,Bingol Police House
5,namibe,-15.20,12.15,71.22,59,0,3.58,AO,1595262873,Hotel Chik Chik Namibe
6,san quintin,30.48,-115.95,76.55,53,0,8.61,MX,1595262875,Old Mill Hotel
7,surt,31.21,16.59,79.50,65,0,5.96,LY,1595262875,City Hotel
8,severodvinsk,64.56,39.83,75.20,38,0,3.00,RU,1595262879,Park Hotel
9,peruibe,-24.32,-47.00,77.86,53,0,1.96,BR,1595262879,Hotel Pousada Xapuri


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [47]:
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)
# Add marker layer ontop of heat map
# name = {"Name":hotel_df["Hotel Name"],"City":hotel_df["City"],"Country":hotel_df["Country"]}
markers = gmaps.marker_layer(locations2, hover_text = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))